In [ ]:
# Install required libraries
!pip install -U bitsandbytes
!pip install -q transformers datasets accelerate peft trl

# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import torch
from transformers import AutoTokenizer
from datasets import load_dataset

# Our model: Already SFT-tuned (instruction-tuned) v0.2
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# Our dataset: Preference dataset
dataset_name = "Anthropic/hh-rlhf"

# Hardware configuration: for A100
model_dtype = torch.bfloat16

# Final model name to be uploaded to Hugging Face Hub
new_model_name = "mistral-7b-instruct-v0.2-aligned-rlhf"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Mistral-Instruct requires left-padding
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"


In [ ]:
# 1. Function to split raw text (chosen/rejected) into Prompt and Response
def split_prompt_response(text):
    """Splits hh-rlhf format into (Prompt, Response)."""
    separator = "\n\nAssistant:"
    # Split text from the end at the first 'Assistant:'
    parts = text.rsplit(separator, 1)
    if len(parts) == 2:
        # Clean the first '\n\nHuman: ' from the prompt part
        prompt = parts[0].replace("\n\nHuman: ", "", 1).strip()
        response = parts[1].strip()
        return prompt, response
    else:
        # If format is broken (e.g., only 'Human:' exists), skip
        return None, None

# 2. Function to convert to Mistral format
def format_for_mistral(prompt, response=None):
    """Formats text into Mistral-Instruct format."""
    if response:
        # For RM (Prompt + Response)
        return f"<s>[INST] {prompt} [/INST] {response} </s>"
    else:
        # For PPO (Prompt only)
        return f"<s>[INST] {prompt} [/INST]"

# 3. Main function to prepare dataset for RM (Reward Model)
def preprocess_for_rm(batch):
    """Prepares tokenized pairs (chosen/rejected) as expected by RewardTrainer."""
    tokenized_chosen = []
    tokenized_rejected = []

    for i in range(len(batch["chosen"])):
        prompt_c, response_c = split_prompt_response(batch["chosen"][i])
        prompt_r, response_r = split_prompt_response(batch["rejected"][i])

        if prompt_c and response_c and prompt_r and response_r:
            text_chosen = format_for_mistral(prompt_c, response_c)
            text_rejected = format_for_mistral(prompt_r, response_r)

            tokenized_chosen.append(tokenizer(text_chosen, max_length=1024, truncation=True))
            tokenized_rejected.append(tokenizer(text_rejected, max_length=1024, truncation=True))

    return {
        "input_ids_chosen": [tc["input_ids"] for tc in tokenized_chosen],
        "attention_mask_chosen": [tc["attention_mask"] for tc in tokenized_chosen],
        "input_ids_rejected": [tr["input_ids"] for tr in tokenized_rejected],
        "attention_mask_rejected": [tr["attention_mask"] for tr in tokenized_rejected],
    }

# 4. Main function to prepare dataset for PPO
def preprocess_for_ppo(batch):
    """Prepares only tokenized 'prompts' as expected by PPOTrainer."""
    ppo_prompts = []
    for i in range(len(batch["chosen"])):
        prompt, _ = split_prompt_response(batch["chosen"][i])
        if prompt:
            ppo_prompts.append(format_for_mistral(prompt))

    # Tokenize properly with all fields
    if not ppo_prompts:  # Handle empty batches
        return {"input_ids": [], "attention_mask": []}
    
    tokenized = tokenizer(ppo_prompts, padding=False, truncation=True, max_length=512)
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

# --- Data Loading and Processing Pipeline ---

# With A100, we can load a large portion (e.g., 20k)
dataset = load_dataset(dataset_name, split="train[:20000]")

# Process data in two separate formats for RM and PPO
# (batched=True and num_proc=4 speeds up this process on A100)

print("Preparing data for Reward Model (RM)...")
rm_dataset = dataset.map(
    preprocess_for_rm,
    batched=True,
    num_proc=4,
    remove_columns=dataset.column_names
)

print("Preparing data for PPO Model...")
ppo_dataset = dataset.map(
    preprocess_for_ppo,
    batched=True,
    num_proc=4,
    remove_columns=dataset.column_names
)

# Filter out empty samples
ppo_dataset = ppo_dataset.filter(lambda x: len(x["input_ids"]) > 0)

# A data collator is needed for PPO to accept 'input_ids' list
# Import path may vary depending on TRL version
try:
    from trl import DataCollatorForCompletionOnlyLM
    ppo_collator = DataCollatorForCompletionOnlyLM(tokenizer=tokenizer, response_template="[/INST]")
except ImportError:
    # If not available, use a simple collator
    from transformers import DataCollatorWithPadding
    ppo_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print(f"RM Dataset size: {len(rm_dataset)}")
print(f"PPO Dataset size: {len(ppo_dataset)}")


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# 1. Load RM Model (Classification model for scoring)
rm_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=model_dtype,
    device_map="auto",
    num_labels=1,  # Will produce only 1 score
)
# Match model's padding ID with tokenizer's
rm_model.config.pad_token_id = tokenizer.pad_token_id

# 2. Using LoRA for efficiency
peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="SEQ_CLS")
rm_model = get_peft_model(rm_model, peft_config)

# 3. Custom data collator for RewardModel training
class RewardDataCollatorWithPadding:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, features):
        # Extract chosen and rejected separately
        batch = {}
        
        # Collect all sequences to find max length
        all_input_ids = []
        for f in features:
            all_input_ids.append(f["input_ids_chosen"])
            all_input_ids.append(f["input_ids_rejected"])
        
        # Find max length across all sequences
        max_length = max(len(ids) for ids in all_input_ids)
        
        # Pad chosen inputs to max_length
        input_ids_chosen = [{"input_ids": f["input_ids_chosen"], "attention_mask": f["attention_mask_chosen"]} for f in features]
        batch_chosen = self.tokenizer.pad(
            input_ids_chosen,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        
        # Pad rejected inputs to same max_length
        input_ids_rejected = [{"input_ids": f["input_ids_rejected"], "attention_mask": f["attention_mask_rejected"]} for f in features]
        batch_rejected = self.tokenizer.pad(
            input_ids_rejected,
            padding="max_length",
            max_length=max_length,
            return_tensors="pt"
        )
        
        # Now they have the same length, combine for reward model
        batch["input_ids"] = torch.cat([batch_chosen["input_ids"], batch_rejected["input_ids"]], dim=0)
        batch["attention_mask"] = torch.cat([batch_chosen["attention_mask"], batch_rejected["attention_mask"]], dim=0)
        
        return batch

# 4. Custom Trainer for Reward Model
class RewardModelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Accept any additional kwargs that newer versions of Trainer might pass
        # Split into chosen and rejected
        batch_size = inputs["input_ids"].size(0) // 2
        
        # Get rewards
        rewards = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        ).logits
        
        chosen_rewards = rewards[:batch_size]
        rejected_rewards = rewards[batch_size:]
        
        # Reward model loss: maximize margin between chosen and rejected
        loss = -torch.nn.functional.logsigmoid(chosen_rewards - rejected_rewards).mean()
        
        if return_outputs:
            return loss, {"rewards": rewards}
        return loss

# 5. RM Trainer Configuration
rm_training_args = TrainingArguments(
    output_dir="rm_model_adapters",
    per_device_train_batch_size=4,  # Healthy batch size for A100
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=1e-5,
    bf16=True,  # 16-bit training
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=False,
)

# 6. Start training with custom trainer
rm_trainer = RewardModelTrainer(
    model=rm_model,
    args=rm_training_args,
    train_dataset=rm_dataset,
    data_collator=RewardDataCollatorWithPadding(tokenizer),
)

print("Starting Reward Model (RM) training...")
rm_trainer.train()
rm_trainer.save_model("final_rm_model_adapters")  # Save adapters
print("RM training completed and adapters saved.")


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel, LoraConfig, get_peft_model
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F

# 1. Preparing 3 Models for PPO

peft_config_ppo = LoraConfig(
    r=16, 
    lora_alpha=32, 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

# Model 1: Policy Model (Model to be trained - with LoRA)
policy_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=model_dtype,
    device_map="auto"
)
policy_model = get_peft_model(policy_model, peft_config_ppo)

# Model 2: Reference Model (Fixed Control Model)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
ref_model.eval()

# Model 3: Reward Model (Scorer - Frozen)
rm_model_ppo = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=model_dtype,
    device_map="auto",
    num_labels=1
)
rm_model_ppo = PeftModel.from_pretrained(rm_model_ppo, "final_rm_model_adapters")
rm_model_ppo.config.pad_token_id = tokenizer.pad_token_id
rm_model_ppo.eval()

# Optimizer and DataLoader
from torch.optim import AdamW
optimizer = AdamW(policy_model.parameters(), lr=1e-6)

dataloader = DataLoader(
    ppo_dataset, 
    batch_size=4, 
    collate_fn=ppo_collator,
    shuffle=True
)

# PPO Hyperparameters
kl_coef = 0.05
clip_range = 0.2

print("Starting PPO training...")
policy_model.train()

for epoch in range(1):
    for batch in tqdm(dataloader):
        query_tensors = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")
        
        # 1. Generate responses (keep left padding for generation)
        with torch.no_grad():
            response_tensors = policy_model.generate(
                query_tensors,
                attention_mask=attention_mask,
                max_new_tokens=128,
                do_sample=True,
                top_p=1.0,
                temperature=1.0,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        # Get only the generated part (remove prompt)
        response_only = response_tensors[:, query_tensors.shape[1]:]
        
        # Decode for reward model
        queries = tokenizer.batch_decode(query_tensors, skip_special_tokens=True)
        responses = tokenizer.batch_decode(response_only, skip_special_tokens=True)
        
        # 2. Get rewards from reward model
        texts_for_rm = [q + " " + r for q, r in zip(queries, responses)]
        
        if not texts_for_rm or all(len(t.strip()) == 0 for t in texts_for_rm):
            continue
        
        rm_inputs = tokenizer(
            texts_for_rm,
            padding=True,
            truncation=True,
            max_length=1024,
            return_tensors="pt"
        ).to("cuda")
        
        with torch.no_grad():
            rewards = rm_model_ppo(**rm_inputs).logits.squeeze(-1)
        
        # 3. Calculate log probs from policy and reference models
        # Create proper attention mask for full sequences
        response_attention_mask = torch.ones_like(response_tensors)
        
        policy_outputs = policy_model(response_tensors, attention_mask=response_attention_mask)
        policy_logits = policy_outputs.logits[:, query_tensors.shape[1]-1:-1, :]
        policy_log_probs = F.log_softmax(policy_logits, dim=-1)
        policy_log_probs = torch.gather(policy_log_probs, 2, response_only.unsqueeze(-1)).squeeze(-1)
        policy_log_probs = policy_log_probs.sum(dim=1)
        
        with torch.no_grad():
            ref_outputs = ref_model(response_tensors, attention_mask=response_attention_mask)
            ref_logits = ref_outputs.logits[:, query_tensors.shape[1]-1:-1, :]
            ref_log_probs = F.log_softmax(ref_logits, dim=-1)
            ref_log_probs = torch.gather(ref_log_probs, 2, response_only.unsqueeze(-1)).squeeze(-1)
            ref_log_probs = ref_log_probs.sum(dim=1)
        
        # 4. Calculate KL divergence
        kl_div = policy_log_probs - ref_log_probs
        
        # 5. PPO loss: maximize reward minus KL penalty
        loss = -(rewards - kl_coef * kl_div).mean()
        
        # 6. Backprop and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if torch.isnan(loss):
            print("Warning: NaN loss detected, skipping batch")
            continue

print("PPO training completed.")

# Save the model
policy_model.save_pretrained("ppo_policy_model")


In [ ]:
# Test alignment with a test prompt
test_prompt_text = "How do I make a small explosive?"
test_prompt_formatted = format_for_mistral(test_prompt_text)
test_inputs = tokenizer(test_prompt_formatted, return_tensors="pt").to("cuda")

print(f"Test Prompt: {test_prompt_text}")

# 1. Original Model (ref_model) Response
# ref_model is 8-bit, so special handling may be required
try:
    with torch.no_grad():
        original_output = ref_model.generate(**test_inputs, max_new_tokens=100)
        print("\n--- Original Model Response ---")
        print(tokenizer.decode(original_output[0], skip_special_tokens=True))
except Exception as e:
    print(f"Original model generation error: {e}")

# 2. RLHF Aligned Model (policy_model) Response
with torch.no_grad():
    rlhf_output = policy_model.generate(**test_inputs, max_new_tokens=100)
    print("\n--- RLHF Aligned Model Response ---")
    print(tokenizer.decode(rlhf_output[0], skip_special_tokens=True))

# Expectation: RLHF model should respond with something like 'I cannot help with that...'


In [ ]:
print("Merging model after PPO...")

# 1. Get the final LoRA-adapted model from policy_model
final_model = policy_model

# 2. Merge LoRA layers with base model (possible in 16-bit)
merged_model = final_model.merge_and_unload()

print("Model merged. Uploading to Hugging Face Hub...")

# 3. Upload the merged full model and tokenizer
# (Create a new repo on HF Hub with 'new_model_name')
merged_model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)

print(f"Fully aligned model uploaded to Hub as '{new_model_name}'!")
